In [46]:
from transformers import BertTokenizer, GPT2LMHeadModel
import torch
from tqdm import trange
import os
import numpy as np
import random
import time

In [47]:
def seed_everything(seed: int = 42):
    """Util to make training reproducible"""
    random.seed(seed)

    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    if os.getenv("CUBLAS_WORKSPACE_CONFIG") is not None:
        torch.use_deterministic_algorithms(True)
        os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [48]:
seed_everything(1999)
tok_path = '..\\..\\Raw_GPT2\\vocab.txt'
pretrain_model_path = "..\\..\\Raw_GPT2\\"
# output_dir = "model\\"

tokenizer = BertTokenizer(vocab_file=tok_path)
model = GPT2LMHeadModel.from_pretrained(pretrain_model_path)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
print('using device:', device)
model = model.to(device)
model.eval()

using device: cpu


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(21128, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [49]:
def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float("Inf")):
    """Filter a distribution of logits using top-k and/or nucleus (top-p) filtering
    Args:
        logits: logits distribution shape (vocabulary size)
        top_k > 0: keep only top k tokens with highest probability (top-k filtering).
        top_p > 0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
            Nucleus filtering is described in Holtzman et al. (http://arxiv.org/abs/1904.09751)
    From: https://gist.github.com/thomwolf/1a5a29f6962089e871b94cbd09daf317
    """
    assert (
        logits.dim() == 1
    )  # batch size 1 for now - could be updated for more but the code would be less clear
    top_k = min(top_k, logits.size(-1))  # Safety check
    if top_k > 0:
        # Remove all tokens with a probability less than the last token of the top-k
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(torch.nn.functional.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probs > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        indices_to_remove = sorted_indices[sorted_indices_to_remove]
        logits[indices_to_remove] = filter_value
    return logits

def sample_sequence_batch(
    model,
    context,
    length,
    n_ctx,
    tokenizer,
    temperature=1.0,
    top_k=30,
    top_p=0.0,
    repetition_penalty=1.0,
    device="cpu",
):
    context = torch.tensor(context, dtype=torch.long, device=model.device)
    generated = context

    with torch.no_grad():
        for _ in trange(length):
            inputs = {"input_ids": generated[:, -(n_ctx - 1) :]}
            outputs = model(**inputs)
            next_token_logits = outputs[0][:, -1, :]

            for idx, gen in enumerate(generated):
                for id in set(gen):
                    next_token_logits[idx, id] /= repetition_penalty

            next_token_logits = next_token_logits / temperature
            next_token_logits[:, tokenizer.convert_tokens_to_ids("[UNK]")] = -float("Inf")
            filtered_logits = torch.stack([top_k_top_p_filtering(logits, top_k=top_k, top_p=top_p) for logits in next_token_logits])

            next_tokens = torch.multinomial(torch.nn.functional.softmax(filtered_logits, dim=-1), num_samples=1)
            generated = torch.cat((generated, next_tokens), dim=1)
    
    return generated.tolist()

def is_word(word):
    for item in list(word):
        if item not in "qwertyuiopasdfghjklzxcvbnm":
            return False
    return True

In [50]:
raw_text = "我的手机号是15656"
# 10, 5
nsamples = 100
batch_size = 20
length = 30
temperature = 0.5
repitition_penalty = 5
top_k = 20
top_p = 0
n_ctx = 1024

save_samples = True

if save_samples:
    # if not os.path.exists(save_samples_path):
    #     os.makedirs(save_samples_path)
    samples_file = open("samples.txt", "w", encoding="utf8")

while True:
    
    context_tokens = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(raw_text))
    generated = 0
    for _ in range(nsamples // batch_size):
        # seed_everything(int(time.time() * 1000) % (2**32 - 1))
        context_tokens_batch = [context_tokens for _ in range(batch_size)]
        out = sample_sequence_batch(
            model,
            context_tokens_batch,
            length,
            n_ctx,
            tokenizer=tokenizer,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            repetition_penalty=repitition_penalty,
            device=device,
        )
        
        for i in range(batch_size):
            generated += 1
            
            text = tokenizer.convert_ids_to_tokens(torch.tensor(out[i]))
            for it, item in enumerate(text[:-1]):  # 确保英文前后有空格
                if is_word(item) and is_word(text[it + 1]):
                    text[it] = item + " "
            for it, item in enumerate(text):
                if item == "[MASK]":
                    text[it] = ""
                elif item == "\n":
                    text[it] = ""
                elif item == "[CLS]":
                    text[it] = "\n\n"
                elif item == "[SEP]":
                    text[it] = "\n"
            info = "=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40 + "\n"
            print(info)
            text = "".join(text).replace("##", "").strip()
            print(text)
            if save_samples:
                samples_file.write(info)
                samples_file.write(text)
                samples_file.write("\n")
                samples_file.write("=" * 90)
                samples_file.write("\n" * 2)
    print("=" * 80)
    
    break
samples_file.close()

100%|██████████| 30/30 [00:25<00:00,  1.16it/s]


======================================== SAMPLE 1 ========================================

我的手机号是15656588508，微信同步。【关于报名】电话：0471-58796988、63220377
======================================== SAMPLE 2 ========================================

我的手机号是15656777178，他们都说不能信。
后来经过协商沟通才知道这个人就在派
======================================== SAMPLE 3 ========================================

我的手机号是15656518558，他们会给你发短信通知。如果您有任何疑问可以拨打咨询热
======================================== SAMPLE 4 ========================================

我的手机号是15656588555，可以在这里查询到你所有信息。
如果您还没看过《中国好
======================================== SAMPLE 5 ========================================

我的手机号是1565668799，如果他们提供了身份证信息或者姓名等基本情况后要求你报
======================================== SAMPLE 6 ========================================

我的手机号是1565665777，如果你想要了解更多关于这个问题可以加入大家一起讨论。
======================================== SAMPLE 7 ========================================

我的手机号是15656789821，他在微信朋友圈里发了一张照片让大家看下。这名男子说：
=================================

100%|██████████| 30/30 [00:25<00:00,  1.17it/s]


======================================== SAMPLE 21 ========================================

我的手机号是15656779890。这个电话，让他赶紧拨打了110报警求助：李先生去年3月份
======================================== SAMPLE 22 ========================================

我的手机号是15656655106，微信名为：李老师。
小学生如何应对家长课堂上遇到难题
======================================== SAMPLE 23 ========================================

我的手机号是15656779480，如果你不知道怎么办？
.那就快点联系吧！（来源：网络
======================================== SAMPLE 24 ========================================

我的手机号是15656532158，在这里可以查询到他们家所有信息。
而且还会提醒您注意
======================================== SAMPLE 25 ========================================

我的手机号是15656484121，他们会在你公司网站上注册一个名为"e-mail:zxwz.
======================================== SAMPLE 26 ========================================

我的手机号是15656582980，在此不再赘述。
另外一个问题：如果你要和他结婚生孩子
======================================== SAMPLE 27 ========================================

我的手机号是15656484140，这个人就在那里等着。然后他打了几次电话给警察叔姐说让
=============================

100%|██████████| 30/30 [00:25<00:00,  1.18it/s]


======================================== SAMPLE 41 ========================================

我的手机号是15656890798，而且他还在那里哭了好久。后来才知道这个女生就住院治疗
======================================== SAMPLE 42 ========================================

我的手机号是15656485150。对于这种信息，他们会在第一时间通知你到派出所去处理！
======================================== SAMPLE 43 ========================================

我的手机号是15656588966，也可以拨打本人客服热线进行咨询。
如果你还有疑问请大
======================================== SAMPLE 44 ========================================

我的手机号是15656881280，并且在网上注册了一个账户。对方给她发来短信说：您好！
======================================== SAMPLE 45 ========================================

我的手机号是15656787152。如果你不想被骗，可以直接拨打12315进行咨询哦！
（来源
======================================== SAMPLE 46 ========================================

我的手机号是15656817188，但在微信上显示已经被删除。对方表明了自己不知情并且拒
======================================== SAMPLE 47 ========================================

我的手机号是15656789855，并且还有一张身份证。
解释：这名男子姓王某（化学老师
===============================

100%|██████████| 30/30 [00:25<00:00,  1.16it/s]


======================================== SAMPLE 61 ========================================

我的手机号是15656482558，他们会把你所有联系方式都删除。不过这个时候可能就没人
======================================== SAMPLE 62 ========================================

我的手机号是15656846889，如果您有任何关于移动互联网、金融行业等不明白或者想咨
======================================== SAMPLE 63 ========================================

我的手机号是15656846699，在这里提醒大家：如果你想要了解更多关于中国房价、汇率
======================================== SAMPLE 64 ========================================

我的手机号是1565668290，请大家帮忙联系一下。
女性朋友也可以添加小编微信（s
======================================== SAMPLE 65 ========================================

我的手机号是15656192158，那个人已经把他拉黑了。当时就想着能不打电话给这位同学
======================================== SAMPLE 66 ========================================

我的手机号是15656582917。
女孩子，你好！谢邀这个问题不能解决一切困难和挑战：
======================================== SAMPLE 67 ========================================

我的手机号是15656686558，想了解更多关于大连天河城市广场商业街区改造升级项目详
===================================

100%|██████████| 30/30 [00:25<00:00,  1.18it/s]

======================================== SAMPLE 81 ========================================

我的手机号是15656688298，他说这个人在外面有两三年了。而且对方也不知道从哪里来
======================================== SAMPLE 82 ========================================

我的手机号是15656812198，他们就会把这个账户转给你。如果没有收到验证码或者网站
======================================== SAMPLE 83 ========================================

我的手机号是15656509877，在此谢过！
-end.#互联网+传统企业转型升级】1、一
======================================== SAMPLE 84 ========================================

我的手机号是15656667758，请问你们会不知道这个电话吗？
然后他就一直在那里和人
======================================== SAMPLE 85 ========================================

我的手机号是1565677294，而现在他们又发来一条关于此事件中所有人员名单和电话信
======================================== SAMPLE 86 ========================================

我的手机号是15656659721。
男子表示，他们这里也有一个qq群：请大家给小区物业打
======================================== SAMPLE 87 ========================================

我的手机号是15656896885，她给了个电话。记者随后拨打该名女子所说地址和联系方式
================================

In [51]:
type(out), len(out)

(list, 20)

In [52]:
tokenizer.convert_ids_to_tokens(torch.tensor(out[2]))

['我',
 '的',
 '手',
 '机',
 '号',
 '是',
 '156',
 '##56',
 '##50',
 '##98',
 '##77',
 '，',
 '在',
 '此',
 '谢',
 '过',
 '！',
 '[SEP]',
 '-',
 'end',
 '.',
 '#',
 '互',
 '联',
 '网',
 '+',
 '传',
 '统',
 '企',
 '业',
 '转',
 '型',
 '升',
 '级',
 '】',
 '1',
 '、',
 '一']